In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
# Load dataset

PlayerStats = pd.read_csv('player_data.csv')
PlayerStats

,Rk,Player,Age,Tm,Pos,GP,G,A,PTS,+/-,...,S,S%,TOI,ATOI,BLK,HIT,FOW,FOL,FO%,year
0,1,Nicholas Abruzzese,23.0,TOR,C,2,0,2,2,1,...,2,0.0,18,8:59,0.0,1,0.0,0.0,NaN,2023
1,2,Noel Acciari,31.0,TOT,C,77,14,9,23,-4,...,102,13.7,1102,14:19,81.0,244,337.0,289.0,53.8,2023
2,2,Noel Acciari,31.0,STL,C,54,10,8,18,-6,...,74,13.5,787,14:35,55.0,168,275.0,239.0,53.5,2023
3,2,Noel Acciari,31.0,TOR,C,23,4,1,5,2,...,28,14.3,315,13:41,26.0,76,62.0,50.0,55.4,2023
4,3,Calen Addison,22.0,MIN,D,62,3,26,29,-17,...,77,3.9,999,16:07,38.0,19,0.0,0.0,NaN,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14627,875,Greg Zanon,29.0,MIN,D,81,2,13,15,-10,...,59,3.4,1812,22:22,196.0,183,0.0,1.0,0.0,2010
14628,876,Henrik Zetterberg,29.0,DET,C,74,23,47,70,12,...,309,7.4,1485,20:04,34.0,44,536.0,543.0,49.7,2010
14629,877,Vladimir Zharkov,22.0,NJD,RW,40,0,10,10,2,...,54,0.0,458,11:26,6.0,22,1.0,3.0,25.0,2010
14630,878,Marek Židlický,32.0,MIN,D,78,6,37,43,-16,...,116,5.2,1885,24:10,128.0,80,0.0,0.0,NaN,2010
